In [1]:
import pandas as pd
import networkx as nx
from pathlib import Path

In [2]:
WORKBOOK_PATH = Path('../../data/assemblies.xlsx')
OUTPUT_PATH = WORKBOOK_PATH

In [3]:
df = pd.read_excel(WORKBOOK_PATH, sheet_name=0)
required_cols = {'ID', 'PredecessorIDs'}

# Build list of edges (predecessor -> task)
edges = []
for _, row in df.iterrows():
    task_id = int(row['ID'])
    preds = str(row['PredecessorIDs']).strip()
    if preds and preds.lower() != 'nan':
        for pred in preds.split(','):
            pred = pred.strip()
            if pred:
                edges.append((int(pred), task_id))

print(f"Collected {len(edges):,} edges")
if edges:
    print("First 10 edges:", edges[:10])  # predecessor → task
else:
    print("⚠️  No edges were collected – double-check the column names/values.")

Collected 418 edges
First 10 edges: [(23, 1), (24, 1), (142, 1), (3, 2), (4, 2), (5, 2), (7, 2), (8, 2), (11, 2), (12, 2)]


In [4]:
# Build directed graph
G = nx.DiGraph()
G.add_edges_from(edges)

# Detect all simple cycles (Johnson's algorithm under the hood)
cycles = list(nx.simple_cycles(G))
print(f'Found {len(cycles)} cycles')

# Flatten cycles to strings like 'A -> B -> C -> A'
cycle_strings = [' → '.join(map(str, c + [c[0]])) for c in cycles]
cycles_df = pd.DataFrame({'Cycle': cycle_strings})
cycles_df.head()

Found 0 cycles


,Cycle
